<a href="https://colab.research.google.com/github/namratabiswas/Agents_Hugging_Face/blob/main/LangGraph_email_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install langgraph langchain_openai

In this notebook we will have my agent James halp me classify emails into SPAM or NOT SPAM categories. If they are SPAM he puts them away in the spam folder. If they are NOT SPAM, he will draft a email for my review.

We’ll implement our helper/agent - named James, email processing system, where he needs to:

1. Read incoming emails
2. Classify them as spam or legitimate
3. Draft a preliminary response for legitimate emails
4. Send information to me when legitimate (printing only)

In [ ]:
# Set the OpenAI API Key environment variable
#%env OPENAI_API_KEY=""
import os
os.environ["OPENAI_API_KEY"] = ""  # <-- your actual key here
print(f"OPENAI_API_KEY is set: {os.environ.get('OPENAI_API_KEY') is not None}")

In [ ]:
#Import necessary libraries
import os
from typing import TypedDict, List, Dict, Any, Optional
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

**Step 1:** Define Our State

  Let’s define what information James needs to track during the email processing workflow:

In [ ]:
class EmailState(TypedDict):
    # The email being processed
    email: Dict[str, Any]  # Contains subject, sender, body, etc.

    # Category of the email (inquiry, complaint, etc.)
    email_category: Optional[str]

    # Reason why the email was marked as spam
    spam_reason: Optional[str]

    # Analysis and decisions
    is_spam: Optional[bool]

    # Response generation
    email_draft: Optional[str]

    # Processing metadata
    messages: List[Dict[str, Any]]  # Track conversation with LLM for analysis

**Step 2:** Define Our Nodes

Now, let’s create the processing functions that will form our nodes:
  

In [ ]:
# Initialize our LLM
model = ChatOpenAI(temperature=0)

def read_email(state: EmailState):
    """James reads and logs the incoming email"""
    email = state["email"]

    # Here we might do some initial preprocessing
    print(f"James is processing an email from {email['sender']} with subject: {email['subject']}")

    # No state changes needed here
    return {}

def classify_email(state: EmailState):
    """James uses an LLM to determine if the email is spam or legitimate"""
    email = state["email"]

    # Prepare our prompt for the LLM
    prompt = f"""
    As James, analyze this email and determine if it is spam or legitimate.

    Email:
    From: {email['sender']}
    Subject: {email['subject']}
    Body: {email['body']}

    First, determine if this email is spam. If it is spam, explain why.
    If it is legitimate, categorize it (inquiry, complaint, thank you, etc.).
    """

    # Call the LLM
    messages = [HumanMessage(content=prompt)]
    response = model.invoke(messages)

    # Simple logic to parse the response
    response_text = response.content.lower()
    is_spam = "spam" in response_text and "not spam" not in response_text

    # Extract a reason if it's spam
    spam_reason = None
    if is_spam and "reason:" in response_text:
        spam_reason = response_text.split("reason:")[1].strip()

    # Determine category if legitimate
    email_category = None
    if not is_spam:
        categories = ["inquiry", "complaint", "thank you", "request", "information"]
        for category in categories:
            if category in response_text:
                email_category = category
                break

    # Update messages for tracking
    new_messages = state.get("messages", []) + [
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": response.content}
    ]

    # Return state updates
    return {
        "is_spam": is_spam,
        "spam_reason": spam_reason,
        "email_category": email_category,
        "messages": new_messages
    }

def handle_spam(state: EmailState):
    """James discards spam email with a note"""
    print(f"James has marked the email as spam. Reason: {state['spam_reason']}")
    print("The email has been moved to the spam folder.")

    # We're done processing this email
    return {}

def draft_response(state: EmailState):
    """James drafts a preliminary response for legitimate emails"""
    email = state["email"]
    category = state["email_category"] or "general"

    # Prepare our prompt for the LLM
    prompt = f"""
    As James the butler, draft a polite preliminary response to this email.

    Email:
    From: {email['sender']}
    Subject: {email['subject']}
    Body: {email['body']}

    This email has been categorized as: {category}

    Draft a brief, professional response that Ms Namrata. can review and personalize before sending.
    """

    # Call the LLM
    messages = [HumanMessage(content=prompt)]
    response = model.invoke(messages)

    # Update messages for tracking
    new_messages = state.get("messages", []) + [
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": response.content}
    ]

    # Return state updates
    return {
        "email_draft": response.content,
        "messages": new_messages
    }

def notify_mr_hugg(state: EmailState):
    """James notifies Ms Namrata(me) about the email and presents the draft response"""
    email = state["email"]

    print("\n" + "="*50)
    print(f"Mam, you've received an email from {email['sender']}.")
    print(f"Subject: {email['subject']}")
    print(f"Category: {state['email_category']}")
    print("\nI've prepared a draft response for your review:")
    print("-"*50)
    print(state["email_draft"])
    print("="*50 + "\n")

    # We're done processing this email
    return {}

**Step 3:** Define Our Routing Logic

We need a function to determine which path to take after classification:

In [ ]:
def route_email(state: EmailState) -> str:
    """Determine the next step based on spam classification"""
    if state["is_spam"]:
        return "spam"
    else:
        return "legitimate"

**Step 4:** Create the StateGraph and Define Edges
Now we connect everything together:

In [ ]:
# Create the graph
email_graph = StateGraph(EmailState)

# Add nodes
email_graph.add_node("read_email", read_email)
email_graph.add_node("classify_email", classify_email)
email_graph.add_node("handle_spam", handle_spam)
email_graph.add_node("draft_response", draft_response)
email_graph.add_node("notify_ms_namrata", notify_ms_namrata)

# Start the edges
email_graph.add_edge(START, "read_email")
# Add edges - defining the flow
email_graph.add_edge("read_email", "classify_email")

# Add conditional branching from classify_email
email_graph.add_conditional_edges(
    "classify_email",
    route_email,
    {
        "spam": "handle_spam",
        "legitimate": "draft_response"
    }
)

# Add the final edges
email_graph.add_edge("handle_spam", END)
email_graph.add_edge("draft_response", "notify_ms_namrata")
email_graph.add_edge("notify_ms_namrata", END)

# Compile the graph
compiled_graph = email_graph.compile()

Step 5: Run the Application

Let’s test our graph with a legitimate email and a spam email:

In [ ]:
 # Example emails for testing
legitimate_email = {
    "sender": "Joker",
    "subject": "Found you Batgirl ! ",
    "body": "Ms. Namrata,I found your secret identity ! I know you're batgirl ! Ther's no denying it, I have proof of that and I'm coming to find you soon. I'll get my revenge. JOKER"
}

spam_email = {
    "sender": "Crypto bro",
    "subject": "The best investment of 2025",
    "body": "Ms Namrata, I just launched an ALT coin and want you to buy some !"
}
# Process legitimate email
print("\nProcessing legitimate email...")
legitimate_result = compiled_graph.invoke({
    "email": legitimate_email,
    "is_spam": None,
    "spam_reason": None,
    "email_category": None,
    "email_draft": None,
    "messages": []
})

# Process spam email
print("\nProcessing spam email...")
spam_result = compiled_graph.invoke({
    "email": spam_email,
    "is_spam": None,
    "spam_reason": None,
    "email_category": None,
    "email_draft": None,
    "messages": []
})

 The output to the above:

Processing legitimate email...
James is processing an email from Joker with subject: Found you Batgirl !


==================================================

Mam, you've received an email from Joker.
Subject: Found you Batgirl !


I've prepared a draft response for your review:

--------------------------------------------------
Subject: Re: Found you Batgirl!

Dear Mr. Joker,

Thank you for reaching out. Your message has been received and noted. Mr. Wayne is currently unavailable, but rest assured, your concerns will be addressed in due course.
ham
Kind regards,

James

Personal Assistant to Ms.Namrata


==================================================


Processing spam email...
James is processing an email from Crypto bro with subject: The best investment of 2025
spam
James has marked the email as spam.
The email has been moved to the spam folder.